In [14]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

import tqdm as tqdm

train = pd.read_csv('/content/drive/MyDrive/데이콘/전력사용량 예측/이동평균_train.csv')
test = pd.read_csv('/content/drive/MyDrive/데이콘/전력사용량 예측/이동평균_test.csv')

In [15]:
train.head()

,num,date_time,target,Temperatures,Wind speed,Humidity,Precipitation,sunshine,비전기냉방설비운영,태양광보유,log_target,month,day,hour,weekday,weekend,date,new_column,열지수,불쾌지수,냉방도일,lunch_time
0,1,2020-06-01 00:00:00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0,9.009454,6,1,0,0,0,2020-06-01,0.0,62.6,63.43008,-8.4,0.0
1,1,2020-06-01 01:00:00,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0,9.004133,6,1,1,0,0,2020-06-01,0.0,63.4,63.56993,-16.7,0.0
2,1,2020-06-01 02:00:00,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0,9.000622,6,1,2,0,0,2020-06-01,0.0,63.4,63.22775,-25.2,0.0
3,1,2020-06-01 03:00:00,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0,8.993404,6,1,3,0,0,2020-06-01,0.0,63.5,62.54339,-34.1,0.0
4,1,2020-06-01 04:00:00,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0,8.992759,6,1,4,0,0,2020-06-01,0.0,62.7,62.39760,-43.1,0.0


In [16]:
test.head()

,num,date_time,Temperatures,Wind speed,Humidity,Precipitation,sunshine,비전기냉방설비운영,태양광보유,month,day,hour,weekday,weekend,date,열지수,불쾌지수,new_column,냉방도일,lunch_time,target,log_target
0,1,2020-08-25 00:00:00,27.800000,1.500000,74.000000,0.0,0.0,0.0,0.0,8,25,0,0,1,2020-08-25,87.4,78.602280,0.0,1.800000,0.0,8667.708709,9.067475
1,1,2020-08-25 01:00:00,27.806027,1.366667,75.017358,0.0,0.0,0.0,0.0,8,25,1,0,1,2020-08-25,87.7,78.746152,0.0,3.606027,0.0,8665.494634,9.067220
2,1,2020-08-25 02:00:00,27.639360,1.233333,76.350691,0.0,0.0,0.0,0.0,8,25,2,0,1,2020-08-25,87.3,78.661547,0.0,5.245387,0.0,8665.041086,9.067167
3,1,2020-08-25 03:00:00,27.300000,1.100000,78.000000,0.0,0.0,0.0,0.0,8,25,3,0,1,2020-08-25,86.4,78.340060,0.0,6.545387,0.0,8648.194145,9.065221
4,1,2020-08-25 04:00:00,26.787947,1.166667,79.965285,0.0,0.0,0.0,0.0,8,25,4,0,1,2020-08-25,84.7,77.770049,0.0,7.333333,0.0,8642.432025,9.064555


In [17]:
final_data = []

for i in train['num'].unique():
  data = pd.concat([train[train['num'] == i], test[test['num'] == i]], axis = 0)
  final_data.append(data)

In [18]:
# 각 모델을 합친 후 모델의 이동평균을 구해주기
for i in final_data:
  i['moving_average'] = i['log_target'].rolling(7).mean()
  i['moving_average'] = i['moving_average'].fillna(i['moving_average'].mean())

In [19]:
a = []
b = []
for i in final_data:
  b.append(i[i['date'] >= '2020-08-25'])
  a.append(i[i['date'] < '2020-08-25'])

In [20]:
# train 정렬하기
for i in range(len(a)):
  a[i] = a[i].sort_values(by = ['date_time'], axis = 0)
  a[i] = a[i].reset_index(drop = True)

# test 정렬하기
for i in range(len(b)):
  b[i] = b[i].sort_values(by = ['date_time'], axis = 0)
  b[i] = b[i].reset_index(drop = True)

In [21]:
# 우리는 모델을 num별로 돌려줄 예정이다.
for i in range(len(a)):
  # month의 데이터 타입을 변경해 준다.
  a[i]['month'] = a[i]['month'].astype('int')
  b[i]['month'] = b[i]['month'].astype('int')

  a[i].loc[(a[i]['hour'] == 12) | (a[i]['hour'] == 13), 'lunch_time'] = 1
  b[i].loc[(b[i]['hour'] == 12) | (b[i]['hour'] == 13), 'lunch_time'] = 1
  a[i] = a[i].fillna(0)
  b[i] = b[i].fillna(0)

# num 별로 주말보다 높은 시간대를 범주화 해주자.
# num = 2
a[1]['working'] = a[1]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 20))  else 0)
b[1]['working'] = b[1]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 20))  else 0)

# num = 4
a[3]['working'] = a[3]['hour'].apply(lambda x:1 if ((8 <= x) & (x <= 21)) else 0)
b[3]['working'] = b[3]['hour'].apply(lambda x:1 if ((8 <= x) & (x <= 21)) else 0)

# num = 5
a[4]['working'] = a[4]['hour'].apply(lambda x:1 if ((5 <= x) & (x <= 18)) else 0)
b[4]['working'] = b[4]['hour'].apply(lambda x:1 if ((5 <= x) & (x <= 18)) else 0)


# num = 6
a[5]['working'] = a[5]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 21)) else 0)
b[5]['working'] = b[5]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 21)) else 0)

# num = 7
a[6]['working'] = a[6]['hour'].apply(lambda x:1 if ((3 <= x) & (x <= 22)) else 0)
b[6]['working'] = b[6]['hour'].apply(lambda x:1 if ((3 <= x) & (x <= 22)) else 0)

# num = 8 7 18
a[7]['working'] = a[7]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 22)) else 0)
b[7]['working'] = b[7]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 22)) else 0)

# num = 10 9 19
a[9]['working'] = a[9]['hour'].apply(lambda x:1 if ((8 <= x) & (x <= 21)) else 0)
b[9]['working'] = b[9]['hour'].apply(lambda x:1 if ((8 <= x) & (x <= 21)) else 0)

# num = 11 10 ~ 20
a[10]['working'] = a[10]['hour'].apply(lambda x:1 if ((9 <= x) & (x <= 21)) else 0)
b[10]['working'] = b[10]['hour'].apply(lambda x:1 if ((9 <= x) & (x <= 21)) else 0)

# num = 12 9 ~ 20
a[11]['working'] = a[11]['hour'].apply(lambda x:1 if ((8 <= x) & (x <= 21)) else 0)
b[11]['working'] = b[11]['hour'].apply(lambda x:1 if ((8 <= x) & (x <= 21)) else 0)

# num = 13 7 17 
a[12]['working'] = a[12]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 19)) else 0)
b[12]['working'] = b[12]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 19)) else 0)

# num = 14 6 19
a[13]['working'] = a[13]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 19)) else 0)
b[13]['working'] = b[13]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 19)) else 0)

# num = 15 7 18
a[14]['working'] = a[14]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 18)) else 0)
b[14]['working'] = b[14]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 18)) else 0)

# num = 16 7 17
a[15]['working'] = a[15]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 19)) else 0)
b[15]['working'] = b[15]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 19)) else 0)

# num = 17 5 21
a[16]['working'] = a[16]['hour'].apply(lambda x:1 if ((5 <= x) & (x <= 22)) else 0)
b[16]['working'] = b[16]['hour'].apply(lambda x:1 if ((5 <= x) & (x <= 22)) else 0)

# num = 18 7 20
a[17]['working'] = a[17]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 22)) else 0)
b[17]['working'] = b[17]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 22)) else 0)

# num = 19 6 8
a[18]['working'] = a[18]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 8)) else 0)
b[18]['working'] = b[18]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 8)) else 0)

# num = 20 6 8
a[19]['working'] = a[19]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 8)) else 0)
b[19]['working'] = b[19]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 8)) else 0)

# num = 21 6 8
a[20]['working'] = a[20]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 8)) else 0)
b[20]['working'] = b[20]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 8)) else 0)

# num = 22 7 19
a[21]['working'] = a[21]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 19)) else 0)
b[21]['working'] = b[21]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 19)) else 0)

# num = 23 
a[22]['working'] = a[22]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 19)) else 0)
b[22]['working'] = b[22]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 19)) else 0)

# num = 24
a[23]['working'] = a[23]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 19)) else 0)
b[23]['working'] = b[23]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 19)) else 0)

# num = 25
a[24]['working'] = a[24]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 19)) else 0)
b[24]['working'] = b[24]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 19)) else 0)

# num = 26
a[25]['working'] = a[25]['hour'].apply(lambda x:1 if ((8 <= x) & (x <= 19)) else 0)
b[25]['working'] = b[25]['hour'].apply(lambda x:1 if ((8 <= x) & (x <= 19)) else 0)

# num = 27
a[26]['working'] = a[26]['hour'].apply(lambda x:1 if ((8 <= x) & (x <= 20)) else 0)
b[26]['working'] = b[26]['hour'].apply(lambda x:1 if ((8 <= x) & (x <= 20)) else 0)

# num = 31 6 18
a[30]['working'] = a[30]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 18)) else 0)
b[30]['working'] = b[30]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 18)) else 0)

# num = 33 8 18
a[32]['working'] = a[32]['hour'].apply(lambda x:1 if ((8 <= x) & (x <= 18)) else 0)
b[32]['working'] = b[32]['hour'].apply(lambda x:1 if ((8 <= x) & (x <= 18)) else 0)

# num = 34 8 18
a[33]['working'] = a[33]['hour'].apply(lambda x:1 if ((18 <= x) & (x <= 19)) else 0)
b[33]['working'] = b[33]['hour'].apply(lambda x:1 if ((18 <= x) & (x <= 19)) else 0)

# num = 35 7 19
a[34]['working'] = a[34]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 19)) else 0)
b[34]['working'] = b[34]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 19)) else 0)

# num = 37 6 18
a[36]['working'] = a[36]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 18)) else 0)
b[36]['working'] = b[36]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 18)) else 0)

# num = 38 7 18
a[37]['working'] = a[37]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 18)) else 0)
b[37]['working'] = b[37]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 18)) else 0)

# num = 39 8 18
a[38]['working'] = a[38]['hour'].apply(lambda x:1 if ((8 <= x) & (x <= 18)) else 0)
b[38]['working'] = b[38]['hour'].apply(lambda x:1 if ((8 <= x) & (x <= 18)) else 0)

# num = 40 10 20
a[39]['working'] = a[39]['hour'].apply(lambda x:1 if ((10 <= x) & (x <= 20)) else 0)
b[39]['working'] = b[39]['hour'].apply(lambda x:1 if ((10 <= x) & (x <= 20)) else 0)

# num = 41 9 21
a[40]['working'] = a[40]['hour'].apply(lambda x:1 if ((9 <= x) & (x <= 21)) else 0)
b[40]['working'] = b[40]['hour'].apply(lambda x:1 if ((9 <= x) & (x <= 21)) else 0)

# num = 42 10 20
a[41]['working'] = a[41]['hour'].apply(lambda x:1 if ((10 <= x) & (x <= 20)) else 0)
b[41]['working'] = b[41]['hour'].apply(lambda x:1 if ((10 <= x) & (x <= 20)) else 0)

# num = 43 7 18
a[42]['working'] = a[42]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 18)) else 0)
b[42]['working'] = b[42]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 18)) else 0)

# num = 44 7 18
a[43]['working'] = a[43]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 18)) else 0)
b[43]['working'] = b[43]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 18)) else 0)

# num = 45 6 17
a[44]['working'] = a[44]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 17)) else 0)
b[44]['working'] = b[44]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 17)) else 0)

# num = 46 4 21
a[45]['working'] = a[45]['hour'].apply(lambda x:1 if ((4 <= x) & (x <= 21)) else 0)
b[45]['working'] = b[45]['hour'].apply(lambda x:1 if ((4 <= x) & (x <= 21)) else 0)

# num = 47 7 21
a[46]['working'] = a[46]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 21)) else 0)
b[46]['working'] = b[46]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 21)) else 0)

# num = 48 7 23
a[47]['working'] = a[47]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 23)) else 0)
b[47]['working'] = b[47]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 23)) else 0)

# num = 49 5 8 
a[48]['working'] = a[48]['hour'].apply(lambda x:1 if ((5 <= x) & (x <= 8)) else 0)
b[48]['working'] = b[48]['hour'].apply(lambda x:1 if ((5 <= x) & (x <= 8)) else 0)

# num = 50 6 8 
a[49]['working'] = a[49]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 8)) else 0)
b[49]['working'] = b[49]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 8)) else 0)

# num = 51 6 8 
a[50]['working'] = a[50]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 8)) else 0)
b[50]['working'] = b[50]['hour'].apply(lambda x:1 if ((6 <= x) & (x <= 8)) else 0)

# num = 52 7 18
a[51]['working'] = a[51]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 16)) else 0)
b[51]['working'] = b[51]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 16)) else 0)

# num = 53 7 18
a[52]['working'] = a[52]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 18)) else 0)
b[52]['working'] = b[52]['hour'].apply(lambda x:1 if ((7 <= x) & (x <= 18)) else 0)

# num = 54 5 19
a[53]['working'] = a[53]['hour'].apply(lambda x:1 if ((5 <= x) & (x <= 19)) else 0)
b[53]['working'] = b[53]['hour'].apply(lambda x:1 if ((5 <= x) & (x <= 19)) else 0)

# num = 55 8 20
a[54]['working'] = a[54]['hour'].apply(lambda x:1 if ((8 <= x) & (x <= 20)) else 0)
b[54]['working'] = b[54]['hour'].apply(lambda x:1 if ((8 <= x) & (x <= 20)) else 0)

# num = 56 8 19
a[55]['working'] = a[55]['hour'].apply(lambda x:1 if ((8 <= x) & (x <= 19)) else 0)
b[55]['working'] = b[55]['hour'].apply(lambda x:1 if ((8 <= x) & (x <= 19)) else 0)

# num = 57 8 17
a[56]['working'] = a[56]['hour'].apply(lambda x:1 if ((8 <= x) & (x <= 17)) else 0)
b[56]['working'] = b[56]['hour'].apply(lambda x:1 if ((8 <= x) & (x <= 17)) else 0)

# num = 58 5 18
a[57]['working'] = a[57]['hour'].apply(lambda x:1 if ((5 <= x) & (x <= 18)) else 0)
b[57]['working'] = b[57]['hour'].apply(lambda x:1 if ((5 <= x) & (x <= 18)) else 0)
# ============================= 시간으로 범주화 끝 ===========================================================

# # 더미화를 원래 하던 방식대로 진행해 주자.
dummies = ['weekday','hour'] # day는 제거해줘야 하는 변수고, month는 달이 오를수록 target변수가 오르므로 그냥 제거해주지 않고 써주도록 하자.
drop_columns_train = ['day','date_time','비전기냉방설비운영','태양광보유','num','target','Wind speed','sunshine','냉방도일','Precipitation','new_column','weekend','date']
for index,train_num in enumerate(a):
  a[index] = pd.get_dummies(data = a[index] , columns = dummies)
  a[index] = a[index].drop(drop_columns_train, axis = 1)

# 더미화를 원래 하던 방식대로 진행해 주자.
dummies = ['weekday','hour'] # day는 제거해줘야 하는 변수고, month는 달이 오를수록 target변수가 오르므로 그냥 제거해주지 않고 써주도록 하자.
drop_columns_test = ['day','date_time','비전기냉방설비운영','태양광보유','num','Wind speed','sunshine','냉방도일','Precipitation','new_column','weekend','date','log_target','target']
for index,test_num in enumerate(b):
  b[index] = pd.get_dummies(data = b[index] , columns = dummies)
  b[index] = b[index].drop(drop_columns_test, axis = 1)

In [22]:
print('train_shape :',a[1].columns)
print('test_shape',b[1].columns)

print('train column length :', len(a[1].columns))
print('test column length :', len(b[1].columns))

train_shape : Index(['Temperatures', 'Humidity', 'log_target', 'month', '열지수', '불쾌지수',
       'lunch_time', 'moving_average', 'working', 'weekday_0', 'weekday_1',
       'hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5', 'hour_6',
       'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12',
       'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
       'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23'],
      dtype='object')
test_shape Index(['Temperatures', 'Humidity', 'month', '열지수', '불쾌지수', 'lunch_time',
       'moving_average', 'working', 'weekday_0', 'weekday_1', 'hour_0',
       'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7',
       'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12', 'hour_13',
       'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18', 'hour_19',
       'hour_20', 'hour_21', 'hour_22', 'hour_23'],
      dtype='object')
train column length : 35
test column length : 34


In [23]:
!pip install vecstack
!pip install catboost
!pip install xgboost

In [24]:
print(len(a[1].columns), len(b[1].columns))

35 34


In [25]:
# 모듈 불러오기
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import KFold, GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso

# STACKING ENSEMBLE을 사용하기 위해서 모델을 불러온다.
import tensorflow as tf
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
from sklearn.model_selection import KFold, GridSearchCV, train_test_split
from vecstack import stacking
from tqdm import tqdm
from sklearn.metrics import r2_score, mean_squared_error

@tf.function
def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(np.log(y), np.log(y_pred)))
def SMAPE(true, pred):
    return np.mean((np.abs(true-pred))/(np.abs(true) + np.abs(pred)))

final_y_pred = []
for i in tqdm(range(len(a))):
  lightgbm = LGBMRegressor(random_state = 42)
  xgboost = XGBRegressor(random_state = 42,tree_method = 'gpu_hist')
  catboost = CatBoostRegressor(random_state =42,task_type = 'GPU')
  rfg = RandomForestRegressor(random_state = 42)
  ridge = Ridge(random_state =42)
  models = [lightgbm, xgboost, catboost, rfg, ridge]

  # random_state를 바꾸어도 모델이 바뀌지는 않는다.
  S_train, S_test = stacking(models, a[i].drop('log_target', axis = 1), a[i]['log_target'], b[i],regression = True, mode = 'oof_pred_bag',metric = rmse, n_folds = 3,random_state = 42)

  lr = LinearRegression()
  lr.fit(S_train, a[i]['log_target'])
  y_pred = lr.predict(S_test)

  final_y_pred.append(y_pred)

y_pred = []
for i in final_y_pred:
  for j in i:
    y_pred.append(j)

y_pred = pd.DataFrame(y_pred, columns = ['y_pred'])

y_pred = y_pred['y_pred'].apply(lambda x:np.exp(x) - 1)

submission['answer'] = y_pred

submission.to_csv('submission.csv', index = False)

print(submission)

Output hidden; open in https://colab.research.google.com to view.

In [28]:
submission = pd.read_csv('/content/drive/MyDrive/데이콘/전력사용량 예측/sample_submission.csv')
submission['answer'] = y_pred

submission.to_csv('submission.csv', index = False)

In [29]:
submission

,num_date_time,answer
0,1 2020-08-25 00,8707.549124
1,1 2020-08-25 01,8705.800391
2,1 2020-08-25 02,8689.518007
3,1 2020-08-25 03,8667.709045
4,1 2020-08-25 04,8657.043989
...,...,...
10075,60 2020-08-31 19,3952.475166
10076,60 2020-08-31 20,3886.283230
10077,60 2020-08-31 21,3772.709733
10078,60 2020-08-31 22,3651.653456


In [ ]:
for i in a:
  print(i.isnull().sum().sum())

for i in b:
  print(i.isnull().sum().sum())
  

In [ ]:
a[3].info()

In [ ]:
b[3].info()